In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.knn import * 

import torch
from torch import nn
from data import data_helper
from data.data_helper import available_datasets
from optimizer.optimizer_helper import get_optim_and_scheduler
from utils.Logger import Logger
import numpy as np
from models.vision_transformer import vit_base_patch16_224, deit_base_patch16_224
from models.convit import convit_base
import timm
import argparse
import os

device = torch.device('cuda:1')

In [ ]:
def get_args():
    parser = argparse.ArgumentParser(description="Script to launch training",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--dataset", default='PACS', help="dataset name")
    parser.add_argument("--source", choices=available_datasets, default=None, help="Source", nargs='+')
    parser.add_argument("--target", choices=available_datasets, help="Target", default=None)
    parser.add_argument("--batch_size", "-b", type=int, default=32, help="Batch size")
    parser.add_argument("--image_size", type=int, default=224, help="Image size")
    # data aug stuff
    parser.add_argument("--min_scale", default=0.8, type=float, help="Minimum scale percent")
    parser.add_argument("--max_scale", default=1.0, type=float, help="Maximum scale percent")
    parser.add_argument("--random_horiz_flip", default=0.5, type=float, help="Chance of random horizontal flip")
    parser.add_argument("--jitter", default=0.4, type=float, help="Color jitter amount")
    parser.add_argument("--tile_random_grayscale", default=0.1, type=float, help="Chance of randomly greyscaling a tile")
    #
    parser.add_argument("--bias_whole_image", default=0.0, type=float, help="If set, will bias the training procedure to show more often the whole image")

    parser.add_argument("--limit_source", default=None, type=int,
                        help="If set, it will limit the number of training samples")
    parser.add_argument("--limit_target", default=None, type=int,
                        help="If set, it will limit the number of testing samples")
    parser.add_argument("--cuda", default=1, type=int, help="Select cuda device")
    parser.add_argument("--target_domain", default='sketch', help="Select target PACS domain")
    parser.add_argument("--classify_only_sane", default=True, type=bool, help="If true, the network will only try to classify the non scrambled images")

    return parser.parse_args(" ".split())

In [ ]:
args = get_args()
if args.dataset == 'PACS':
    args.n_classes = 7
    if args.target_domain == 'photo':
        args.source = ['art_painting', 'cartoon', 'sketch']
        args.target = 'photo'
    elif args.target_domain == 'sketch':
        args.source = ['art_painting', 'cartoon', 'photo']
        args.target = 'sketch'
    elif args.target_domain == 'cartoon':
        args.source = ['art_painting', 'photo', 'sketch']
        args.target = 'cartoon'
    else:
        args.source = ['photo', 'cartoon', 'sketch']
        args.target = 'art_painting'
        
args.batch_size = args.batch_size // 3 + 1

In [ ]:
model = vit_base_patch16_224(pretrained=True, num_classes=args.n_classes).to(device)

In [ ]:
with torch.cuda.device(1):
    classifier = KNNClassifier(model=model)

In [ ]:
with torch.cuda.device(1):
    source_loaders, val_loader = data_helper.get_train_dataloader(args)

In [ ]:
with torch.cuda.device(1):
    classifier._loadTrainDataset(source_loaders)